# 節點部分處理細節---科技報橘網頁爬蟲

In [1]:
import requests
import lxml
import pdfkit
import os
import bs4
from bs4 import BeautifulSoup as bs
from IPython.display import display, HTML

### 讀入網址

In [2]:
url = ['https://buzzorange.com/techorange/2017/06/20/start-from-zero-learning-big-data/',
      'https://buzzorange.com/techorange/2017/08/30/how-to-work-on-ai/',
      'https://buzzorange.com/techorange/2017/08/21/the-best-ai-lesson/']

### 載入網頁

In [3]:
page = requests.get(url[0])
article = bs(page.text,'lxml')

# 查看網頁原貌(JS、CSS可能影響整個notebook表現
# display(HTML(str(article)))

# 單純查看格式整理後的HTML輸出
# print(str(article.prettify()))

### 細部整理

In [4]:
# 擷取到有全部主內容的最小節點
content = article.find('article')

In [5]:
# 去除主內容內的雜訊(e.g.廣告)
content.find(class_='fb-count').decompose()
content.find(class_='byline').decompose()
# 迴圈去除同類雜訊
for item in content.find_all('img',class_='aligncenter'):
    item.decompose()

In [6]:
# 從某特定節點後開始去除同類雜訊
startDecompose = False
for item in content.find(class_='entry-content').children:
    if item==content.find(id='share_group_btns'):
        startDecompose = True
    if startDecompose:
        if type(item)!=bs4.NavigableString:
            item.decompose()

In [7]:
# 取出標題部分
title = str(content.find('h1').contents[0])

filetitle = title[:]
# 暫存HTML和後續PDF的檔名不能包含部分符號之移除
for ch in '*|\\:"<>?/':
    filetitle = filetitle.replace(ch, "")

### 建立新網頁節點樹

In [8]:
# 建立樣板
newPage = bs('<html><head><title></title><meta charset="UTF-8"></head><body></body></html>','lxml')

# 插入網頁標題、文章
newPage.title.insert(len(newPage.title.contents),title)
newPage.body.insert(len(newPage.body.contents),content)

### 檔案匯出

In [9]:
#建立HTML檔
with open(title+'.html','w',encoding='utf-8') as file:
    file.write(newPage.prettify())
    
#讀入HTML檔，產生PDF
if os.path.exists(filetitle+'.pdf'):
    os.remove(filetitle+'.pdf')
pdfkit.from_file(title+'.html', title+'.pdf')

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


True